<a href="https://colab.research.google.com/github/uskhbs/Machine_learning/blob/main/5_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# 트리의 앙상블

# 앙상블 학습 1. 랜덤 포레스트(random forest)
# 돌아보기 -> 분류 : 샘플을 몇 개의 클래스 중 하나로 분류하는 문제, 회귀 : 임의의 어떤 숫자를 예측하는 문제

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)


from sklearn.model_selection import cross_validate   # 교차 검증 수행
from sklearn.ensemble import RandomForestClassifier  # 기본 100개 결정트리
rf = RandomForestClassifier(n_jobs=-1, random_state=42)

scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)  # return_train_score = True ->  검증 점수 뿐만 아니라 훈련세트에 대한 점수도 반환
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [39]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [40]:
# 자체적으로 모델을 평가하는 점수 -> OOB(out of bag)
# oob_score 매개변수를 True : 랜덤 포레스트는 각 결정 트리의 OOB 점수를 평균하여 출력
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


In [41]:
# 앙상블 학습 2. 엑스트라 트리(Extra Trees)
# 엑스트라 트리의 회귀 버전 : ExtraTreesRegressor
# 랜덤 포레스트와의 차이점 : 부트스트랩 샘플을 사용하지 않는다.

from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [42]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


In [43]:
# 앙상블 학습 3. 그레이디언트 부스팅(gradient boosting)
# 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
# 일반적으로 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수 있다. 하지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느리다.

from sklearn.ensemble import GradientBoostingClassifier # 깊이가 3인 결정 트리를 100개 사용
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42) # learning_rate 의 기본 값 0.1
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015
0.9464595437171814 0.8780082549788999


In [44]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


In [45]:
# 앙상블 학습 4. 히스토그램 기반 그레이디언트 부스팅(Histogream-based Gradient Boosting)
# 정형 제이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높은 알고리즘

from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [46]:
# pernutation_importance()
# 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지 계산
# 특성 중요도(importance), 평균(importances_mean), 표준 편자(importances_std)

from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)

# n_repeats : 랜덤하게 섞을 횟수 (기본값 5)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

result = permutation_importance(hgb, test_input, test_target, n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]
[0.05969231 0.20238462 0.049     ]


In [47]:
hgb.score(test_input, test_target) 
# 결과 : 87% 정확도

0.8723076923076923

In [52]:
# XGBoost 라이브러리 : 그레디언트 부스팅 알고리즘을 구현한 라이브러리
from xgboost import XGBClassifier
# tree_mothod = 'hist': 히스토그램 기반 그레이디언트 부스팅 사용가능
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [54]:
# LightGBM : 마이크로소포트에서 만든 히스토그램 기반 그레이디언트 부스팅 라이브러리
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9338079582727165 0.8789710890649293
